# Loading Minigrid envs to Storm 

## Environments

First we import the necessary packages

In [64]:
from minigrid.envs.lavagap import LavaGapEnv
from stormvogel import bird, show, ModelType


### 1. LavaGap Env

Initially we define the lava gap and some necessary information we need form the env

In [65]:
env = LavaGapEnv(5)
env.reset()
init_dir = env.agent_dir
init_pos = tuple(map(int, env.agent_pos))
init = (init_pos, init_dir)


Then we define the functions label (given a state returns the correct lable as a string), available actions (for a state gives all available actions as a 2d list of strings), and delta (defines the new state fora given state, action pair)

In [66]:
def labels(s: bird.State):
    cell_type = "None"
    cell = env.grid.get(s[0][0], s[0][1])
    if cell is not None: 
        cell_type = cell.type
    return f"Position: {s[0]}, Direction: {s[1]}, Cell Type: {cell_type}"

def available_actions(s: bird.State):
    return [["right"], ["left"], ["forward"]]

def delta(s: bird.State, a: bird.Action):
    curr_state = env.grid.get(s[0][0], s[0][1])
    if curr_state is not None and (curr_state.type == "lava" or curr_state.type == "goal"): 
        return [(1, s)]
    
    if a[0] == "right": 
        result = (s[0], (s[1] + 1) % 4)
        return [(1, result)]
    if a[0] == "left":
        if s[1]-1 < 0: 
            result = (s[0], s[1]-1 + 4)
        else: 
            result = (s[0], s[1]-1)
        return [(1, result)]
    if a[0] == "forward":
        env.agent_pos = s[0]
        env.agent_dir = s[1]
        fwd_pos = env.front_pos
        # print(f"forward pos in delta: {fwd_pos}")
        fwd_cell = env.grid.get(fwd_pos[0], fwd_pos[1])
        result = (tuple(map(int, fwd_pos)), s[1])
        # print(f"result in delta: {result}")

        if fwd_cell is None or fwd_cell.can_overlap():
            return [(1,result)]
        if fwd_cell is not None and (fwd_cell.type == "goal" or fwd_cell.type == "lava"):
                return [(1,result)]

Then all is left is to build the stormvogel model and display it

In [67]:
model = bird.build_bird(
                    delta=delta,
                    init=init, 
                    labels=labels, 
                    available_actions=available_actions, 
                    modeltype=ModelType.MDP
                    )
visual = show(model, show_editor=True)

Output()

In [71]:
print(len(model.states))

20
